In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [3]:
def zero_pad(X,pad):
    """
    X:(m,x_h,x_w,c)
    pad:integer
    """
    X_pad=np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),'constant',constant_values=0)
    return X_pad

In [5]:
x = np.random.randn(4, 3, 3, 2)
x_pad = zero_pad(x, 1)
print(x.shape)
print(x_pad.shape)

(4, 3, 3, 2)
(4, 5, 5, 2)


In [ ]:
def conv_single_step(a_slice_prev,W,b):
    s=a_slice_prev*W+b
    z=np.sum(s)
    return z

In [9]:
def conv_forward(A_prev,W,b,hparameters):
    """
    A_prev:(m,n_h_prev,n_w_prev,n_c_prev)
    W:(f,f,n_c_prev,n_c)
    b:(1,1,1,n_c)
    hparamters: dictionary containing "strid" and "pad"
    
    Return:
    Z:conv output,(m,n_h,n_w,n_c)
    cache: cache of values needed for the conv_backward()function
    """
    (m,n_H_prev,n_W_prev,n_C_prev)=A_prev.shape
    (f,f,n_C_prev,n_C)=W.shape
    stride=hparameters["stride"]
    pad = hparameters["pad"]
    
    n_H=int((n_H_prev-f+2*pad)/stride)+1
    n_W=int((n_W_prev-f+2*pad)/stride)+1
    Z=np.zeros(m,n_H,n_W,n_C)
    
    A_prev_pad=zero_pad(A_prev,pad)
    for i in range(m):
        a_prev_pad=A_prev_pad[i]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start=h*stride
                    vert_end=vert_start+f
                    horiz_start=w*stride
                    horiz_end=horize+f
                    a_prev_slice=z_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]
                    Z[i,h,w,c]=conv_single_step(a_prev_slice,W[:,:,:,c],b[:,:,:c])
    cache=(A_prev,W,b,hparameters)
    return Z,cache
    

In [11]:
def pool_forward(A_prev,hparameters,mode="max"):
    """
    A_prev(m,n_H_prev,n_W_prev,n_C_prev)
    hparameters: dictionary of ("stride","f")
    mode:"max"or"average"
    Returns:
    A(m,n_H,n_W,n_C)
    
    """
    (m,n_H_prev,n_W_prev,n_C_prev)=A_prev.shape
    stride=hparameters["stride"]
    f=hparameters["f"]
    
    n_H=int((n_H_prev-f)/stride)+1
    n_W=int((n_W_prev-f)/stride)+1
    A=np.zeros(m,n_H,n_W,n_C)
    for i in range(m):
        a_prev=A_prev[i]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start=h*stride
                    vert_end=vert_start+f
                    horiz_start=w*stride
                    horiz_end=horiz_start+f
                    a_prev_slice=a_prev[vert_start:vert_end,horiz_start:horiz_end,c]
                    if mode=="max":
                        A[i,h,w,c]=np.max(a_prev_slice)
                    if mode=="average":
                        A[i,h,w,c]=np.mean(a_prev_slice)
    cache=(A_prev,hparameters)
    return Z,cache

In [12]:
def conv_backward(dZ, cache):
    """
    dZ:shape(m,n_H,n_W,n_C)
    cache(A_prev,W,b,hparameters)
    
    Returns:
    dA_prev
    dW(f,f,n_C_prev,n_C)
    db(1,1,1,n_C)
    
    """
    (A_prev,W,b,hparameters) = cache
    (m,n_H_prev,n_W_prev,n_C_prev) = A_prev.shape
    (f,f,n_C_prev,n_C)=W.shape
    (m,n_H,n_W,n_C)=dZ.shape
    
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    dA_prev=np.zeros(A_prev.shape)
    dW=np.zeros(W.shape)
    db=np.zeros(b.shape)
    
    A_prev_pad=zero_pad(A_prev,pad)
    dA_prev_pad=zero_pad(dA_prev,pad)
    
    for i in range(m):
        a_prev=A_prev_pad[i]
        da_prev=dA_prev_pad[i]
        for h in range(n_H):
            for w in range(n_W):
                vert_start=h*stride
                vert_end=vert_start+f
                horiz_start=w*stride
                horiz_end=horiz_start+f
                
                a_prev_slice = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]
                da_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]+=W[:,:,:,c]*dZ[i,h,w,c]
                dW[:,:,:,c]+=a_prev_slice*dZ[i,h,w,c]
                db[:,:,:,c]+=dZ[:,:,:,c]
        dA_prev[i,:,:,:]=da_prev_pad[pad:-pad,pad:-pad,:]
        
    return dA_prev,dW,db
    

In [13]:
def create_mask_form_window(x):
    mask=(x==np.max(x))
    return mask


In [14]:
def distribute_value(dz,shape):
    (n_H,n_W)=shape
    average=n_H*n_W
    a=np.ones(shape)*average
    return a

In [15]:
def pool_backward(dA,cache,mode="max"):
    
    (A_prev,hparameters)=cache
    stride=hparameters["stride"]
    f=hparameters["f"]
    m,n_H_prev,n_W_prev,n_C_prev=A_prev.shape
    m,n_H,n_W,n_C=dA.shape
    dA_prev=np.zeros(A_prev.shape)
    
    for i in range(m):
        a_prev=A_prev[i]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start=h*stride
                    vert_end=vert_start+f
                    horiz_start=w*stride
                    horiz_end=horiz_start+f
                    if mode=="max":
                        a_prev_slice=a_prev[vert_start:vert_end,horiz_start:horiz_end,c]
                        mask=create_mask_from_window(a_prev_slice)
                        dA_prev[i,vert_start:vert_end,horiz_start:horiz_end,c]+=mask*dA[i,vert_start,horiz_start,c]
                    if mode=="average":
                        da=dA[i,vert_start,horiz_start,c]
                        shape=(f,f)
                        dA_prev[i,vert_start:vert_end,horiz_start:horiz_end,c]+=distribute_value(da,shape)
        return dA_prev
    

In [ ]:
gent